############### crie uma tabela de produtos já unindo categorias, marca(brand), e quantidade total de estoque em todas as lojas

In [0]:
%sql
USE CATALOG bikestore;

In [0]:
# Definir pastas do projetos em variaveis para facilitar
bronze_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/resource/origem'
resource_path_volume = '/Volumes/bikestore/logistics/bikestore_rescource/origem/'

In [0]:
# criar varias tabelas temporárias de forma prática
bronze_map = {
    "tmp_bronze_brands":      f"{bronze_path}/brands/",
    "tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_details/",
    "tmp_bronze_orders":      f"{bronze_path}/orders/",
    "tmp_bronze_products":    f"{bronze_path}/products/",
    "tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    "tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    "tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
%sql
WITH cte_stock AS (
  SELECT 
    product_id,
    SUM(quantity) AS total_stock
  FROM tmp_bronze_stocks
  GROUP BY product_id
)

SELECT
  p.product_id,
  p.product_name,
  b.brand_name,
  c.category_name,
  p.model_year,
  p.list_price,
  s.total_stock
FROM tmp_bronze_products AS p
LEFT JOIN tmp_bronze_categories AS c 
  ON c.category_id = p.category_id
LEFT JOIN tmp_bronze_brands AS b 
  ON b.brand_id = p.brand_id
LEFT JOIN cte_stock AS s
  ON s.product_id = p.product_id;


In [0]:
 WITH cte_stock AS (
    SELECT 
      product_id,
      SUM(quantity) AS total_stock
    FROM tmp_bronze_stocks
    GROUP BY product_id
  )
 
 select * from  WITH cte_stock;

In [0]:
df_product_silver = spark.sql("""
  WITH cte_stock AS (
    SELECT 
      product_id,
      SUM(quantity) AS total_stock
    FROM tmp_bronze_stocks
    GROUP BY product_id
  )

  SELECT
    p.product_id,
    p.product_name,
    b.brand_name,
    c.category_name,
    p.model_year,
    p.list_price,
    s.total_stock
  FROM tmp_bronze_products AS p
  LEFT JOIN tmp_bronze_categories AS c 
    ON c.category_id = p.category_id
  LEFT JOIN tmp_bronze_brands AS b 
    ON b.brand_id = p.brand_id
  LEFT JOIN cte_stock AS s
    ON s.product_id = p.product_id;
""")



## salvar em Delta na silver
df_product_silver.write\
                 .mode("overwrite") \
                 .format("delta") \
                 .option("mergeSchema", "true") \
                 .save(f"{silver_path}/product")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS  bikestore.logistics.silver_products
LOCATION "abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/silver/product";